### Notes about this working page
This page is a working notebook were it was edited according to the requirements. This was used mainly for dataset creation and based on Pytorch <br>
We batch extracted faces using batch MTCNN. In addition this page was used for batch augmentation for creating our dataset. <br>
Please note that after the dataset creation we shifted to TensorFlow instead of Pytorch. <br>

Code inspired from the following website
https://www.kaggle.com/timesler/guide-to-mtcnn-in-facenet-pytorch
https://machinelearningmastery.com/

In [ ]:
from facenet_pytorch import MTCNN
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import scipy

### Group photo Face Detection

In [ ]:
# Create face detector
detector = MTCNN(image_size=182, margin=20, keep_all=True, post_process=False)

# Load a single image and display
v_cap = cv2.VideoCapture('../coursework/images/raw/78/IMG_6990.jpg')
success, frame = v_cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = Image.fromarray(frame)
#frame = frame.rotate(275)

plt.figure(figsize=(18, 6))
plt.imshow(frame)
plt.axis('off')
plt.show()

# Detect face
faces = detector(frame)
print('Face Detected ...')

# Visualize
fig, axes = plt.subplots(1, len(faces))
for face, ax in zip(faces, axes):
    ax.imshow(face.permute(1, 2, 0).int().numpy())
    ax.axis('off')
fig.show()

#save_paths = [f'../coursework/images/processed/78/image300_{i}.jpg' for i in range(100,200)]
#detector(frame, save_path=save_paths);
#print('Image Saved ...')

### Video Batch Processing

In [ ]:
detector = MTCNN(image_size=182, margin=20, select_largest=False, post_process=False)

# Load a vide8
v_cap = cv2.VideoCapture('../coursework/images/Class/jpeg/IMG_6819.jpg')
v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Loop through video
batch_size = 58
frames = []
boxes = []
landmarks = []
view_frames = []
view_boxes = []
view_landmarks = []
for _ in tqdm(range(v_len)):
    
    # Load frame
    success, frame = v_cap.read()
    if not success:
        continue
        
    # Add to batch, resizing for speed
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    frame = frame.resize([int(f * 0.25) for f in frame.size])
    frames.append(frame)
    
    # When batch is full, detect faces and reset batch list
    
    if len(frames) >= batch_size:
        batch_boxes, _, batch_landmarks = detector.detect(frames, landmarks=True)
        boxes.extend(batch_boxes)
        landmarks.extend(batch_landmarks)
        
        view_frames.append(frames[-1])
        view_boxes.append(boxes[-1])
        view_landmarks.append(landmarks[-1])
        
        frames = []

save_paths = [f'../coursework/images/processed/78/image1000_{i}.jpg' for i in range(len(frames))]
detector(frames, save_path=save_paths);
print('Image Saved')

#Visualize
fig, ax = plt.subplots(2, 2, figsize=(18, 12))
for i in range(4):
    ax[int(i / 2), i % 2].imshow(view_frames[i])
    ax[int(i / 2), i % 2].axis('off')
    for box, landmark in zip(view_boxes[i], view_landmarks[i]):
        ax[int(i / 2), i % 2].scatter(*np.meshgrid(box[[0, 2]], box[[1, 3]]), s=8)
        ax[int(i / 2), i % 2].scatter(landmark[:, 0], landmark[:, 1], s=6)

### Image Augmentation

In [3]:
import cv2
import numpy as np
from skimage import io, exposure, img_as_ubyte
from skimage.transform import rotate
from skimage.util import random_noise
import matplotlib.pyplot as plt
import random
import os

### Augmentation Functions

In [ ]:
def v_flip(image):
    return np.flipud(image)

def h_flip(image):
    return  np.fliplr(image)

def add_noise(image):
    return random_noise(image)

def blur(image):
    return cv2.GaussianBlur(image, (9,9),0)

def gamma(image, gamma=0.5):
    return exposure.adjust_gamma(image, gamma) 

def anticlockwise_rotation(image):
    angle= random.randint(0,180)
    return rotate(image, angle)

def clockwise_rotation(image):
    angle= random.randint(0,180)
    return rotate(image, -angle)

### Batch Augmentation

In [ ]:
Augmentation = { 'Horizontal Flip': h_flip, 
                    'Vertical Flip': v_flip,
                    'Noise': add_noise,
                    'Gamma': gamma } 

# Raw & augmented images paths
rawPath="C:/Users/Tareq/Desktop/vision/coursework/images/processed/unknown 3"
augmentedPath="C:/Users/Tareq/Desktop/vision/coursework/images/processed/unknown 3" 

images=[] 

# Append to images array     
for im in os.listdir(rawPath):  
    images.append(os.path.join(rawPath,im))

# Number of images needed
images_to_generate=80  

i=1                       

while i<=images_to_generate:    
    image=random.choice(images)
    original_image = io.imread(image)
    augmented=None

    n = 0      
    count = random.randint(1, len(Augmentation)) #choose random number of transformation to apply on the image
    
    # Random Functions to augment
    while n <= count:
        key = random.choice(list(Augmentation))
        augmented = Augmentation[key](original_image)
        n = n + 1
        
    new_image_path= "%s/augmented_image_%s.jpg" %(augmentedPath, i)
    augmented = img_as_ubyte(augmented)
    
    # RGB conversion before saving
    augmented=cv2.cvtColor(augmented, cv2.COLOR_BGR2RGB) 
    cv2.imwrite(new_image_path, augmented) 
    i =i+1